In [7]:
import numpy as np
import pandas as pd
from collections import Counter
from Data import load_data

rng = np.random.default_rng(42)

players, points, rebounds, assists, weights = load_data("./nbasalariespoints.csv", "Player", "PS/G▼", "TRB", "AST", "SALARY")

In [8]:
# Create a population of num_teams x team_size, with random values from 0-last index
# Each individual represents a team, and each team has 15 players
# **has uniform distribution**
def initializePopulation(pop_size, team_size):
    return rng.integers(0, len(weights) - 1, (pop_size, team_size), dtype=int)

def calculateValue(individual, obj):
    return np.sum(obj[individual])

def calculateWeight(individual, weights):
    return np.sum(weights[individual])


# Change so that instead of objectives being constrained, it calculates the distance between the team's stats
# and an idea point that the user inputs. Then use 1/dist as the fitness.
# Individuals that are far away from the point will have low fitness, and ones that are close
# will have high fitness. Then, use the best as parents for the next population.
# Include a step of calculating how many times a certian player appears in the top (10? 15?) teams
# and add them into the distrubution that many times
# This leads to a non-uniform distribution, which allows for better players to be picked more often

# Sets the fitness to -inf if the weight exceeds the knapsack capacity, or if there are duplicate players (can't draft the same player twice)
# Otherwise, sets the fitness to the value of the team
# Since there is a limit on how many players can be in each team, don't use value to weight ratio as the fitness
def calculateFitness(individual, weights, capacity, ideal_point, objectives):
    val_list = []
    total = 0

    for objective in objectives:
        val_list.append(calculateValue(individual, objective))
    
    for i in range(len(ideal_point)):
        total += (ideal_point[i] - val_list[i]) ** 2
    dist = np.sqrt(total)

    # Use 1/dist as fitness
    fitness = 100/dist

    # Can't choose the same player twice
    if (len(set(individual)) < 15) or (calculateWeight(individual, weights) > capacity):
        fitness = -np.inf
    return fitness

def addCopies(weights, objectives, counter, objective_names):
    df = pd.DataFrame()
    df["Weights"] = weights
    for i in range(len(objectives)):
        if not objective_names:
            df[f"Objective_{i + 1}"] = objectives[i]
        else:
            df[objective_names[i]] = objectives[i]
    
    # can use index for keys and use 15 * index + inner index to make length faster (maybe)
    for key in counter.keys():
        for _ in range(counter[key]):
            df.loc[len(df)] = df.iloc[key].values

    return df

def adjustDistribution(capacity, weights, population, ideal_point, objectives, objective_names, counter):
    fitness = []
    distribution = np.ones(len(weights)) * 1 / len(weights)
    for individual in population:
        fitness.append(calculateFitness(individual, weights, capacity, ideal_point, objectives))
        
    indices = np.argsort(fitness)
    population = population[indices][::-1]
    fitness = np.sort(fitness)[::-1]

    # Calculate how many times each player from the fit population apears
    # then add that many copies into the distribution before sampling again
    # This (effectively) leads to weighting each player, allowing for the better players to be chosen more often 
    # Use counter from collections 
    for i in range(15):
        counter += Counter(population[i])

    for index, count in counter.items():
        distribution[index] = count / len(distribution)
        
    return distribution


def Sample(distribution, pop_size, team_size):
    length = len(distribution)
    population = np.zeros((pop_size, team_size), dtype=int)
    distribution = distribution / distribution.sum()    

    for i in range(pop_size):
        population[i] = rng.choice(length, team_size, p = distribution)
    return population

def knapSackGenetic(capacity, weights, training_length, pop_size, team_size, ideal_point, objectives, objective_names = None):
    population = initializePopulation(pop_size, team_size)
    counter = Counter()

    for i in range(training_length):
        distribution = adjustDistribution(capacity, weights, population, ideal_point, objectives, objective_names, counter)
        population = Sample(distribution, pop_size, team_size)

    valid = []
    for individual in population:
        if calculateFitness(individual, weights, capacity, ideal_point, objectives) == -np.inf: continue
        valid.append(individual)

    valid = np.array(valid)
    if len(valid) == 0:
        return -1   
    indices = np.argsort([val for val in [calculateFitness(individual, weights, capacity, ideal_point, objectives) for ind in valid]])
    print(f"Fitness of best choice: {calculateFitness(valid[indices[-1]], weights, capacity, ideal_point, objectives)}")
    print(f"Value of objective 1: {calculateValue(valid[indices[-1]], objectives[0])}")
    print(f"Value of objective 2: {calculateValue(valid[indices[-1]], objectives[1])}")
    print(f"Value of objective 3: {calculateValue(valid[indices[-1]], objectives[2])}")
    return valid[indices[-1]]


In [ ]:
ideal_point = [400, 100, 100]
capacity = 150_000_000
training_length = 1000
pop_size = 100
team_size = 15

individual = knapSackGenetic(capacity, weights, training_length, pop_size, team_size, ideal_point, [points, rebounds, assists], ["Points", "Rebounds", "Assists"])
print(np.sort(individual))
print(players[individual])


Fitness of best choice: 0.567361024206463
Value of objective 1: 230.29999999999998
Value of objective 2: 68.6
Value of objective 3: 64.2
[  8  10  12  13  20  22  59  69  76  81  82 136 150 160 183]
15
['John Wall' 'Carmelo Anthony' 'Marcus Thornton' 'Rodney Hood'
 'Emmanuel Mudiay' 'Darrell Arthur' 'Trevor Ariza' 'Kyrie Irving'
 'Zach LaVine' 'Taj Gibson' 'Blake Griffin' 'Russell Westbrook'
 "D'Angelo Russell" 'Isaiah Thomas' 'Rodney Stuckey']
